In [24]:
import requests
import os
import json
import pandas as pd

In [5]:
# example url
# https://interactives.apelections.org/election-results/data-live/2024-11-05/results/races/AL/20241105AL0/detail.json

def get_state_url(state_abbr: str) -> str:
    date = "2024-11-05"
    compressed_date = date.replace("-",'')
    url = f"https://interactives.apelections.org/election-results/data-live/{date}/results/races/{state_abbr}/{compressed_date}{state_abbr}0/detail.json"
    return url

In [6]:
state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

In [8]:
state_urls = [get_state_url(state) for state in state_abbreviations]
state_urls[:2]

['https://interactives.apelections.org/election-results/data-live/2024-11-05/results/races/AL/20241105AL0/detail.json',
 'https://interactives.apelections.org/election-results/data-live/2024-11-05/results/races/AK/20241105AK0/detail.json']

In [10]:
# Folder to save the JSON files
output_folder = 'state_json_files'
os.makedirs(output_folder, exist_ok=True)

# Loop through the state URLs, fetch the JSON data, and save to files
for url, abbr in zip(state_urls, state_abbreviations):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors

        # Save the JSON content to a file
        file_path = os.path.join(output_folder, f'{abbr}.json')
        with open(file_path, 'w') as file:
            file.write(response.text)

        print(f'Successfully saved {abbr}.json')

    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred for {abbr}: {http_err}')
    except Exception as err:
        print(f'An error occurred for {abbr}: {err}')

Successfully saved AL.json
Successfully saved AK.json
Successfully saved AZ.json
Successfully saved AR.json
Successfully saved CA.json
Successfully saved CO.json
Successfully saved CT.json
Successfully saved DE.json
Successfully saved FL.json
Successfully saved GA.json
Successfully saved HI.json
Successfully saved ID.json
Successfully saved IL.json
Successfully saved IN.json
Successfully saved IA.json
Successfully saved KS.json
Successfully saved KY.json
Successfully saved LA.json
Successfully saved ME.json
Successfully saved MD.json
Successfully saved MA.json
Successfully saved MI.json
Successfully saved MN.json
Successfully saved MS.json
Successfully saved MO.json
Successfully saved MT.json
Successfully saved NE.json
Successfully saved NV.json
Successfully saved NH.json
Successfully saved NJ.json
Successfully saved NM.json
Successfully saved NY.json
Successfully saved NC.json
Successfully saved ND.json
Successfully saved OH.json
Successfully saved OK.json
Successfully saved OR.json
S

In [11]:
trump_id = "8639"
harris_id = "64984"

In [20]:
# Folder containing the JSON files
input_folder = "state_json_files"
state_results = []

# Loop through each JSON file in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(input_folder, filename)
        try:
            with open(file_path, "r") as file:
                data = json.load(file)

            # Extract the "summary" key
            summary = data.get("summary", {})

            # Extract the "candidate" array within "summary"
            candidate_array = summary.get("candidates", [])

            # Output the candidate array
            state_abbr = filename.split(".")[0]

            for candidates in candidate_array:
                if candidates["candidateID"] == trump_id:
                    trump_votes = candidates["voteCount"]
                elif candidates["candidateID"] == harris_id:
                    harris_votes = candidates["voteCount"]

            payload = {
                "state": state_abbr,
                "trump_votes": trump_votes,
                "harris_votes": harris_votes,
            }
            state_results.append(payload)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {filename}: {e}")
        except Exception as e:
            print(f"An error occurred while processing file {filename}: {e}")

In [26]:
df = pd.DataFrame(state_results)
df.to_csv("results.csv")